In [1]:
import numpy as np

def get_grad(Y):
    I = np.zeros(Y.shape)
    
    for i in range(I.shape[0]):
        if i == 0:
            I[i] = Y[i + 1] - Y[i]
        elif i == I.shape[0] - 1:
            I[i] = Y[i] - Y[i - 1]
        else:
            I[i] = Y[i + 1] - Y[i - 1]
    
    return I

def get_mask(I, direction):
    
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            if i > 0:
                add = I[i - 1][j]
                if j > 0:
                    add = min(add, I[i - 1][j - 1])
                if j < I.shape[1] - 1:
                    add = min(add, I[i - 1][j + 1])
                I[i][j] += add
    
    mask = np.zeros(I.shape)
    
    cur_id = 0
    
    for  j in range(0, I.shape[1]):
        if I[-1, j] < I[-1, cur_id]:
            cur_id = j
    
    for i in range(I.shape[0] - 1, 0, -1):
        mask[i][cur_id] = 1
        new_cur_id = max(cur_id - 1, 0)
        for j in range(max(cur_id - 1, 0), min(I.shape[1] - 1, cur_id + 1)):
                if I[i - 1][j] < I[i - 1][new_cur_id]:
                    new_cur_id = j
        cur_id = new_cur_id
    
    return mask

def delete_mask(img, mask):
    
    img_res = np.zeros((img.shape[0], img.shape[1] - 1, 3))
    
    for k in range(3):
        for i in range(img.shape[0]):
            index = mask[i].argmax()
            img_res[i,:,k] = np.hstack((img[i, :index, k], img[i, index + 1:, k]))
    
    return img_res

def seam_carve(img, work_type, mask=None):
    
    work_type = work_type.split(' ')
    
    if work_type[0] == 'vertical':
        img = img.transpose((1, 0, 2))
    
    Y = 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]
    
    I_x = get_grad(Y)
    I_y = get_grad(Y.T).T
    
    I = (I_x ** 2 + I_y ** 2) ** 0.5
    
    mask = get_mask(I, work_type[0])
    
    img = delete_mask(img, mask)
    
    if work_type[0] == 'vertical':
        img = img.transpose((1, 0, 2))
        mask = mask.T
    
    return img, None, mask

In [20]:
from os.path import join
from os import environ
import numpy as np

def check_test(data_dir):
    from pickle import load
    output_dir = join(data_dir, 'output')
    gt_dir = join(data_dir, 'gt')
    correct = 0
    with open(join(output_dir, 'output_seams'), 'rb') as fout, \
         open(join(gt_dir, 'seams'), 'rb') as fgt:
        for i in range(8):
            A = np.array(load(fout))
            B = np.array(load(fgt))
            print((A - B)[-10:])
            print(A[-10:])
            print(B[-10:])
            break
            if load(fout) == load(fgt):
                correct += 1
    res = 'Ok %d/8' % correct
    if environ.get('CHECKER'):
        print(res)
    return res

In [21]:
check_test('public_tests/01_test_img_check')

ValueError: operands could not be broadcast together with shapes (474,2) (475,2) 